In [33]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [34]:
# Mathematical/data cleaning tools
import numpy as np
import pandas as pd
from collections import Counter
from pathlib import Path

#sklearn
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix

In [35]:
# Fast AI
import fastai
from fastai import *
from fastai.text import *
from fastai.text.data import *

### Reading in Data to Pandas

In [36]:
twitter = pd.read_csv('../data/full-corpus.csv')

In [37]:
# twitter = twitter.drop(columns=['Topic', 'TweetId', 'TweetDate'], axis=1)
twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


### Reading in Data to FastAI

In [38]:
path = Path('/home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data')
path

PosixPath('/home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data')

In [39]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'full-corpus.csv', text_cols='TweetText', label_cols=1)
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'full-corpus.csv', vocab=data_lm.train_ds.vocab, bs=32)

#### Train One Cycle

In [40]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.121075,4.107698,0.281622,02:12


In [41]:
learn.save_encoder('ft_enc_1_cycles')

## Sentence Completion

In [42]:
learn.predict("This is a review about", n_words=10)

"This is a review about Steve Jobs ' trends at the time in"

In [43]:
learn.predict("I love my new iphone, the features are", n_words=10)

'I love my new iphone, the features are very amazing , glad this , and hey i have'

#### Train 100 Cycles

In [12]:
for _ in range(0, 100):
    learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.099711,3.555772,0.348735,04:29


epoch,train_loss,valid_loss,accuracy,time
0,3.706372,3.374535,0.368266,05:26


epoch,train_loss,valid_loss,accuracy,time
0,3.431626,3.263700,0.382589,04:56


epoch,train_loss,valid_loss,accuracy,time
0,3.253056,3.188880,0.392820,04:39


epoch,train_loss,valid_loss,accuracy,time
0,3.080493,3.128003,0.400037,04:55


...

epoch,train_loss,valid_loss,accuracy,time
0,1.323419,3.321657,0.451600,04:29


epoch,train_loss,valid_loss,accuracy,time
0,1.302502,3.328306,0.452567,04:17


epoch,train_loss,valid_loss,accuracy,time
0,1.276721,3.337661,0.453013,04:12


epoch,train_loss,valid_loss,accuracy,time
0,1.324262,3.335401,0.452530,04:48


epoch,train_loss,valid_loss,accuracy,time
0,1.330728,3.328266,0.452232,04:19


#### Save Encoder

In [44]:
learn.save_encoder('ft_enc_100_cycles')

In [45]:
learn.predict("I love my new iphone, the features are", n_words=10)

'I love my new iphone, the features are even worse than i never seen before it was right'

In [46]:
learn.predict("Steve Ballmer is", n_words=3)

'Steve Ballmer is a visual director'

In [47]:
learn.predict("Albert Einstein is ", n_words=4)

'Albert Einstein is  Ice Cream'

In [48]:
learn.predict("I love", n_words=10)

'I love you to love me is Steve King and'

## Classification

In [49]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc_100_cycles')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (4090 items)
x: TextList
xxbos xxunk,xxbos xxunk,xxbos xxunk,xxbos xxunk,xxbos xxunk
y: CategoryList
apple,microsoft,apple,microsoft,twitter
Path: /home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data;

Valid: LabelList (1023 items)
x: TextList
xxbos xxunk,xxbos xxunk,xxbos negative,xxbos xxunk,xxbos xxunk
y: CategoryList
google,apple,microsoft,twitter,microsoft
Path: /home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(4592, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(4592, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1

In [50]:
data_clas.show_batch()

text,target
xxbos xxunk,apple
xxbos xxunk,twitter
xxbos xxunk,apple
xxbos xxunk,microsoft
xxbos xxunk,apple


In [51]:
learn.fit(50)

epoch,train_loss,valid_loss,accuracy,time
0,1.390298,1.345903,0.308895,00:29
1,1.368812,1.344534,0.320626,00:28
2,1.372785,1.343912,0.308895,00:27
3,1.364598,1.344602,0.308895,00:31
4,1.363309,1.347038,0.308895,00:27
5,1.361609,1.342911,0.308895,00:29
6,1.360380,1.351021,0.308895,00:31
7,1.362143,1.343487,0.320626,00:34
8,1.364113,1.343870,0.308895,00:28
9,1.368534,1.342152,0.308895,00:26


In [32]:
learn.predict("I prefer my iphone")

(Category microsoft, tensor(2), tensor([0.2214, 0.2624, 0.2642, 0.2520]))